In [1]:
import time
import math
import inspect
from dataclasses import dataclass
from typing import Any, Callable, List, Optional, Tuple
import random

import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import functional as F
from torchvision.ops import box_iou, distance_box_iou

## Box IoU

In [8]:
def _batched_box_iou(boxes1: Tensor, boxes2: Tensor) -> Tensor:
    """
    Return intersection-over-union (Jaccard index) between a batch of two sets of boxes.

    Both sets of boxes are expected to be in ``(x1, y1, x2, y2)`` format with
    ``0 <= x1 < x2`` and ``0 <= y1 < y2``.

    Args:
        boxes1 (Tensor[..., N, 4]): batch of first set of boxes
        boxes2 (Tensor[..., M, 4]): batch of second set of boxes

    Returns:
        Tensor[..., N, M]: each NxM matrix containing the pairwise IoU values for every element in boxes1 & boxes2 pair
    """
    area1 = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])  # [..., N]
    area2 = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])  # [..., M]
    lt = torch.max(boxes1[..., None, :2], boxes2[..., None, :, :2])  # [..., N, M, 2]
    rb = torch.min(boxes1[..., None, 2:], boxes2[..., None, :, 2:])  # [..., N, M, 2]
    wh = (rb - lt).clamp(min=0)  # [..., N, M, 2]
    inter = wh[..., 0] * wh[..., 1]  # [..., N, M]
    union = area1[..., None] + area2[..., None, :] - inter  # [..., N, M]
    iou = inter / union  # [..., N, M]
    return iou

In [10]:
B = 100_000

preds = torch.tensor(
    [
        [296.55, 93.96, 314.97, 152.79],
        [328.94, 97.05, 342.49, 122.98],
        [356.62, 95.47, 372.33, 147.55],
    ]
)
preds = preds.unsqueeze(0).expand(B, -1, -1)

target = torch.tensor(
    [
        [300.00, 100.00, 315.00, 150.00],
        [330.00, 100.00, 350.00, 125.00],
        [350.00, 100.00, 375.00, 150.00],
    ]
)
target = target.unsqueeze(0).expand(B, -1, -1)

In [13]:
t0 = time.time()
for i in range(B):
    iou = box_iou(preds[i], target[i])
t1 = time.time()
print(iou)  # tensor([[0.6898, 0.0000, 0.0000],
            #         [0.0000, 0.5086, 0.0000],
            #         [0.0000, 0.0000, 0.5654]])
print("torchvision.ops.box_iou:", t1-t0)

torchvision.ops.box_iou: 4.528836488723755


In [15]:
t0 = time.time()
for p, t in zip(preds, target):
    iou = box_iou(p, t)
t1 = time.time()
print(iou)  # tensor([[0.6898, 0.0000, 0.0000],
            #         [0.0000, 0.5086, 0.0000],
            #         [0.0000, 0.0000, 0.5654]])
print("torchvision.ops.box_iou:", t1-t0)

tensor([[0.6898, 0.0000, 0.0000],
        [0.0000, 0.5086, 0.0000],
        [0.0000, 0.0000, 0.5654]])
torchvision.ops.box_iou: 4.543814420700073


In [58]:
t0 = time.time()
iou = _batched_box_iou(preds, target)
t1 = time.time()
print(iou.shape)
print(iou[11])  # tensor([[0.6898, 0.0000, 0.0000],
                #         [0.0000, 0.5086, 0.0000],
                #         [0.0000, 0.0000, 0.5654]])
print("_batched_box_iou:", t1-t0)

torch.Size([100000, 3, 3])
tensor([[0.6898, 0.0000, 0.0000],
        [0.0000, 0.5086, 0.0000],
        [0.0000, 0.0000, 0.5654]])
_batched_box_iou: 0.003949642181396484


## Distance Box IoU

In [9]:
def _batched_distance_box_iou(boxes1: Tensor, boxes2: Tensor, eps: float = 1e-7) -> Tensor:
    """
    Return distance intersection-over-union (Jaccard index) between a batch of two sets of boxes.
    Both sets of boxes are expected to be in ``(x1, y1, x2, y2)`` format with ``0 <= x1 < x2`` and ``0 <= y1 < y2``.
    Args:
        boxes1 (Tensor[..., N, 4]): batch of first set of boxes
        boxes2 (Tensor[..., M, 4]): batch of second set of boxes
    Returns:
        Tensor[..., N, M]: each NxM matrix containing the pairwise IoU values for every element in boxes1 & boxes2 pair
    """
    iou = _batched_box_iou(boxes1, boxes2)
    lti = torch.min(boxes1[..., None, :2], boxes2[..., None, :, :2])
    rbi = torch.max(boxes1[..., None, 2:], boxes2[..., None, :, 2:])
    whi = (rbi - lti).clamp(min=0)
    diagonal_distance_squared = (whi[..., 0] ** 2) + (whi[..., 1] ** 2) + eps
    # Centers of boxes
    cx_1 = (boxes1[..., 0] + boxes1[..., 2]) / 2
    cy_1 = (boxes1[..., 1] + boxes1[..., 3]) / 2
    cx_2 = (boxes2[..., 0] + boxes2[..., 2]) / 2
    cy_2 = (boxes2[..., 1] + boxes2[..., 3]) / 2
    # Distance between boxes' centers squared
    centers_distance_squared = ((cx_1[..., None] - cx_2[..., None, :]) ** 2) + ((cy_1[..., None] - cy_2[..., None, :]) ** 2)
    return iou - (centers_distance_squared / diagonal_distance_squared)

In [2]:
B = 100_000

preds = torch.tensor(
    [
        [296.55, 93.96, 314.97, 152.79],
        [328.94, 97.05, 342.49, 122.98],
        [356.62, 95.47, 372.33, 147.55],
    ]
)
preds = preds.unsqueeze(0).expand(B, -1, -1)

target = torch.tensor(
    [
        [300.00, 100.00, 315.00, 150.00],
        [330.00, 100.00, 350.00, 125.00],
        [350.00, 100.00, 375.00, 150.00],
    ]
)
target = target.unsqueeze(0).expand(B, -1, -1)

In [3]:
t0 = time.time()
for i in range(B):
    iou = distance_box_iou(preds[i], target[i])
t1 = time.time()
print(iou)  # tensor([[ 0.6883, -0.2043, -0.3351],
            #         [-0.2214,  0.4886, -0.1913],
            #         [-0.3971, -0.1510,  0.5609]])
print("torchvision.ops.distance_box_iou:", t1-t0)

tensor([[ 0.6883, -0.2043, -0.3351],
        [-0.2214,  0.4886, -0.1913],
        [-0.3971, -0.1510,  0.5609]])
torchvision.ops.distance_box_iou: 11.184566736221313


In [4]:
t0 = time.time()
for p, t in zip(preds, target):
    iou = distance_box_iou(p, t)
t1 = time.time()
print(iou)  # tensor([[ 0.6883, -0.2043, -0.3351],
            #         [-0.2214,  0.4886, -0.1913],
            #         [-0.3971, -0.1510,  0.5609]])
print("torchvision.ops.box_iou:", t1-t0)

tensor([[ 0.6883, -0.2043, -0.3351],
        [-0.2214,  0.4886, -0.1913],
        [-0.3971, -0.1510,  0.5609]])
torchvision.ops.box_iou: 11.260282039642334


In [10]:
t0 = time.time()
iou = _batched_distance_box_iou(preds, target)
t1 = time.time()
print(iou.shape)
print(iou[11])  # tensor([[ 0.6883, -0.2043, -0.3351],
                #         [-0.2214,  0.4886, -0.1913],
                #         [-0.3971, -0.1510,  0.5609]])
print("_batched_distance_box_iou:", t1-t0)

torch.Size([100000, 3, 3])
tensor([[ 0.6883, -0.2043, -0.3351],
        [-0.2214,  0.4886, -0.1913],
        [-0.3971, -0.1510,  0.5609]])
_batched_box_iou: 0.012221097946166992
